# Лабораторна робота № 5

## ІП-14 Бабіч Денис

---

# Підготовчий етап

## Імпортування необхідних модулів

## Створення допоміжних функцій

---

# Основний етап

## Застосування на прикладі зображення

## Застосування на прикладі відео